In [10]:
# This notebook is for creating a training sample and a separate testing sample
# that includes lepton pt,eta,phi, met and ht from event
# file all_jets_fullRun2_v1.parquet

In [11]:
import awkward as ak
import numba
import numpy as np
import pandas as pd
import awkward as ak
import os
import h5py
import vector
vector.register_numba()
vector.register_awkward()


a=22
b=a+2
#import matplotlib.pyplot as plt
#from matplotlib.colors import LogNorm
#import mplhep as hep
#hep.style.use(hep.style.ROOT)

In [12]:
main_dir= "configs/HH4b/out_parton_matching_lastB_fulldataset/"
filename = f"{main_dir}/GluGlutoHHto4B.parquet"
df = ak.from_parquet(filename)

In [13]:
print(ak.count(df.JetGood))
print(ak.count(df.JetGoodMatched))
print(ak.count(df.JetGoodHiggs))
print(ak.count(df.JetGoodHiggsMatched))


21798560
15279606
18159400
15161874


In [14]:
jets = ak.with_name(df.JetGood, name="Momentum4D")
jets_higgs = ak.with_name(df.JetGoodHiggs, name="Momentum4D")
# b_quarks = ak.with_name(df.bQuark, name="Momentum4D")

jets
len(jets)
print(jets.fields)
print(jets.matched[a:b])
print(jets.prov[a:b])

print(jets_higgs.matched[a:b]
        )
print(jets_higgs.prov[a:b])

['pt', 'eta', 'phi', 'btag', 'ptPnetRegNeutrino', 'matched', 'prov']
[[True, True, True, True], [True, True, True, True, False]]
[[1, 1, 2, 2], [1, 2, 2, 1, -1]]
[[True, True, True, True], [True, True, True, True]]
[[1, 1, 2, 2], [1, 2, 2, 1]]


In [15]:
frac_train = 0.8

ntot = ak.count(jets)
print(ntot)
ntot=len(jets)
index_train_max = int(np.ceil(frac_train*ntot))
ntot

21798560


648550

In [16]:
jets_train = jets[:index_train_max]

jets_test = jets[index_train_max:]

print(index_train_max)
print(ak.count(jets_train), ak.count(jets_test))
print(len(jets_train), len(jets_test))
idx_train = np.random.choice(ntot, index_train_max, replace=False)
print(idx_train)


518840
17439800 4358760
518840 129710
[561454 623587 575069 ... 208141 454411 152810]


In [17]:
def create_groups(file):
    file.create_group("TARGETS/h1") # higgs 1 -> b1 b2
    file.create_group("TARGETS/h2") # higgs 2 -> b3 b4
    file.create_group("INPUTS")
    file.create_group("INPUTS/Source")
    file.create_group("INPUTS/ht")
    return file

def create_targets(file, particle, jets):
    multiindex = ak.zip([ak.local_index(jets, i) for i in range(jets.ndim)])

    higgs_targets = {1: ["b1", "b2"], 2: ["b3", "b4"]}

    for j in [1,2]:
        if particle == f"h{j}":
            mask = jets.prov == j # H->b1b2
            multiindex2 = multiindex[mask]
            print(multiindex2)

            b1_array = []
            b2_array = []

            for index,i in enumerate(multiindex2):
                if len(i) == 0:
                    b1_array.append(-1)
                    b2_array.append(-1)
                elif len(i) == 1:
                    b1_array.append(i[0].tolist()[1])
                    b2_array.append(-1)
                elif len(i) == 2:
                    b1_array.append(i[0].tolist()[1])
                    b2_array.append(i[1].tolist()[1])

            file.create_dataset(f"TARGETS/h{j}/{higgs_targets[j][0]}", np.shape(b1_array), dtype='int64', data=b1_array)
            file.create_dataset(f"TARGETS/h{j}/{higgs_targets[j][1]}", np.shape(b2_array), dtype='int64', data=b2_array)


def create_inputs(file, jets):
    pt_array = ak.to_numpy(ak.fill_none(ak.pad_none(jets.pt, 16, clip=True), 0))
    mask = ~(pt_array == 0)
    mask_ds = file.create_dataset("INPUTS/Jet/MASK", np.shape(mask), dtype='bool', data=mask)
    pt_ds = file.create_dataset("INPUTS/Jet/pt", np.shape(pt_array), dtype='float32', data=pt_array)

    ptPnetRegNeutrino_array = ak.to_numpy(ak.fill_none(ak.pad_none(jets.ptPnetRegNeutrino, 16, clip=True), 0))
    ptPnetRegNeutrino_ds = file.create_dataset("INPUTS/Jet/ptPnetRegNeutrino", np.shape(ptPnetRegNeutrino_array), dtype='float32', data=ptPnetRegNeutrino_array)

    phi_array = ak.to_numpy(ak.fill_none(ak.pad_none(jets.phi, 16, clip=True), 0))
    phi_ds = file.create_dataset("INPUTS/Jet/phi", np.shape(phi_array), dtype='float32', data=phi_array)

    eta_array = ak.to_numpy(ak.fill_none(ak.pad_none(jets.eta, 16, clip=True), 0))
    eta_ds = file.create_dataset("INPUTS/Jet/eta", np.shape(eta_array), dtype='float32', data=eta_array)

    btag = ak.to_numpy(ak.fill_none(ak.pad_none(jets.btag, 16, clip=True), 0))
    btag_ds = file.create_dataset("INPUTS/Jet/btag", np.shape(btag), dtype='float32', data=btag)

    # Fill ht
    pt_array = ak.to_numpy(ak.fill_none(ak.pad_none(jets.pt, 15, clip=True), 0))
    ht_array = np.sum(pt_array, axis=1)
    ht_ds = file.create_dataset("INPUTS/ht/ht", np.shape(ht_array), dtype='float32', data=ht_array)

    # Fill ht
    ptPnetRegNeutrino_array = ak.to_numpy(ak.fill_none(ak.pad_none(jets.ptPnetRegNeutrino, 15, clip=True), 0))
    htPNetRegNeutrino_array = np.sum(ptPnetRegNeutrino_array, axis=1)
    htPNetRegNeutrino_ds = file.create_dataset("INPUTS/ht/htPNetRegNeutrino", np.shape(htPNetRegNeutrino_array), dtype='float32', data=htPNetRegNeutrino_array)

In [18]:
mask_fullymatched = ak.sum(jets_test.matched == True, axis=1)>=4
jets_test_fullymatched = jets_test[mask_fullymatched]
print(len(jets_test_fullymatched))

mask_fullymatched = ak.sum(jets.matched == True, axis=1)>=4
jets_fullymatched = jets[mask_fullymatched]
print(len(jets_fullymatched))

120498
602531


In [19]:
# higgs = jets_test_fullymatched[jets_test_fullymatched.prov == 1]
# mask_match = ak.num(higgs) == 2

# w_or_t_jets = jets_test_fullymatched[(jets_test_fullymatched.prov == 5)|(jets_test_fullymatched.prov == 2)]
# mask_match = mask_match & (ak.num(w_or_t_jets) == 3)

# lep_top = jets_test_fullymatched[jets_test_fullymatched.prov == 3]
# mask_match = mask_match & (ak.num(lep_top) == 1)

# jets_test_fullymatched = jets_test_fullymatched[mask_match]
# jets_test_fullymatched

In [21]:
# Prepare files for inputs and targets and overwrite them
# remove old file
# os.remove(f"{main_dir}/output_all.h5")
test_file = h5py.File(f"{main_dir}/output_test.h5", "w")
test_file = create_groups(test_file)

In [22]:
# create_targets(test_file, "h1", jets_test_fullymatched)
# create_targets(test_file, "h2", jets_test_fullymatched)
create_targets(test_file, "h1", jets)
create_targets(test_file, "h2", jets)


[[(0, 1), (0, 3)], [(1, 1), (1, 2)], ... ), (648548, 3)], [(648549, 2), (648549, 3)]]


KeyboardInterrupt: 

In [ ]:
# Create input arrays in the files
# create_inputs(test_file, jets_test_fullymatched)
create_inputs(test_file, jets)

In [ ]:
test_file

<HDF5 file "output_all.h5" (mode r+)>

In [ ]:
# Print the index of b-jets
print(test_file["TARGETS"]["h1"]["b1"][a:b])
print(test_file["TARGETS"]["h1"]["b2"][a:b])
print(test_file["TARGETS"]["h2"]["b3"][a:b])
print(test_file["TARGETS"]["h2"]["b4"][a:b])
print(jets.matched[a:b])
print(jets.prov[a:b])

print(jets_higgs.matched[a:b]
        )
print(jets_higgs.prov[a:b])
# print(b_quarks.matched[a:b])
# print(b_quarks.prov[a:b])

#deltaR between b-quarks and jets
# deltaR=ak.cartesian({"a":jets_higgs,"b":b_quarks}, nested=True)
# deltaR=deltaR.a.deltaR(deltaR.b)

# print(deltaR[a:b])

[0 0]
[3 4]
[1 2]
[2 3]
[[True, True, True, True], [True, False, True, True, True, False]]
[[1, 2, 2, 1], [1, -1, 2, 2, 1, -1]]
[[True, True, True, True], [True, False, True, True]]
[[1, 2, 2, 1], [1, -1, 2, 2]]
[[1, 1, 2, 2], [1, 2, 2, 1]]
[[[0.926, 0.00689, 2.98, 2.73], [2.72, 2.85, ... 3.09], [2.38, 1.19, 0.0403, 2.15]]]


In [ ]:
# Print the btag score of jets
test_file["INPUTS"]["Jet"]["btag"][a:b]
test_file["INPUTS"]["Source"]


OSError: [Errno 116] Can't synchronously read data (file read failed: time = Thu Feb 15 15:31:50 2024
, filename = 'configs/HH4b/out_parton_matching_lastB_new//output_all.h5', file descriptor = 88, errno = 116, error message = 'Stale file handle', buf = 0x5632aabecf08, total read size = 65536, bytes this sub-read = 65536, bytes actually read = 18446744073709551615, offset = 0)

In [ ]:
# test_file.close()